<a href="https://colab.research.google.com/github/thefenriswolf/ML/blob/master/Gen7.1Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt

# check for active GPU
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
train_dir = 'drive/My Drive/PROSTATE-DIAGNOSIS/train/'
test_dir = 'drive/My Drive/PROSTATE-DIAGNOSIS/test/'
backup_dir = 'drive/My Drive/models/'

pos_train_dir = os.path.join(train_dir, 'pos')
neg_train_dir = os.path.join(train_dir, 'neg')
pos_test_dir = os.path.join(test_dir, 'pos')
neg_test_dir = os.path.join(test_dir, 'neg')

num_pos_tr = len(os.listdir(pos_train_dir))
num_neg_tr = len(os.listdir(neg_train_dir))
num_pos_val = len(os.listdir(pos_test_dir))
num_neg_val = len(os.listdir(neg_test_dir))

total_train = num_pos_tr + num_neg_tr
total_val = num_pos_val + num_neg_val

print('total positive training images:', num_pos_tr)
print('total negative training images:', num_neg_tr)
print('total positive test images:', num_pos_val)
print('total negative test images:', num_neg_val)
print("--------------------------------------")
print("Total training images:", total_train)
print("Total test images:", total_val)

total positive training images: 11210
total negative training images: 1840
total positive test images: 3108
total negative test images: 226
--------------------------------------
Total training images: 13050
Total test images: 3334


In [5]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 160
IMG_WIDTH = 160
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)


train_image_generator = ImageDataGenerator(rescale=1./255, 
                                           rotation_range=6,
                                           #horizontal_flip=True,
                                           vertical_flip=True,
                                           zoom_range=0.3)

test_image_generator = ImageDataGenerator(rescale=1./255)

print("Train:")
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary',
                                                           interpolation='nearest')
print("Test:")
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=test_dir,
                                                              shuffle=True,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary',
                                                              interpolation='nearest')

Train:
Found 13050 images belonging to 2 classes.
Test:
Found 3334 images belonging to 2 classes.


In [0]:
#restore previously trained model

restored_model = tf.keras.models.load_model('drive/My Drive/models/cnn_revX.h5')
restored_model.summary()

In [6]:
# Create the base model from MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

#base_model.trainable = False # dont train base model
base_model.trainable = True # make base model trainable

# Print Number of Layers in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer in the base model onwards
fine_tune_at = 100

# Freeze all the layers before `fine_tune_at`
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

base_model.summary()

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)


9412608/9406464 [==============================] - 0s 0us/step
Number of layers in the base model:  155
Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][

In [7]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,863,873
Non-trainable params: 395,392
_________________________________________________________________


In [9]:
history = model.fit(train_data_gen,
                    epochs=epochs,
                    steps_per_epoch=total_train//(batch_size*1.5),
                    validation_data=test_data_gen,
                    validation_steps=total_val//batch_size,
                    verbose=True)

Epoch 1/15
67/67 [==============================] - 6276s 94s/step - loss: 0.3129 - accuracy: 0.8822 - val_loss: 1.2750 - val_accuracy: 0.3939
Epoch 2/15
67/67 [==============================] - 1665s 25s/step - loss: 0.1995 - accuracy: 0.9244 - val_loss: 1.0813 - val_accuracy: 0.4603
Epoch 3/15
67/67 [==============================] - 71s 1s/step - loss: 0.1399 - accuracy: 0.9498 - val_loss: 1.4979 - val_accuracy: 0.4889
Epoch 4/15
67/67 [==============================] - 72s 1s/step - loss: 0.0991 - accuracy: 0.9648 - val_loss: 1.6952 - val_accuracy: 0.4002
Epoch 5/15
67/67 [==============================] - 70s 1s/step - loss: 0.0927 - accuracy: 0.9657 - val_loss: 1.1008 - val_accuracy: 0.6031
Epoch 6/15
67/67 [==============================] - 70s 1s/step - loss: 0.0742 - accuracy: 0.9708 - val_loss: 1.8098 - val_accuracy: 0.5057
Epoch 7/15
67/67 [==============================] - 70s 1s/step - loss: 0.0578 - accuracy: 0.9791 - val_loss: 2.6763 - val_accuracy: 0.2939
Epoch 8/15
67/

In [0]:
# Save model as HDF5
model.save('drive/My Drive/models/cnn_rev7-3.h5')
# model.save('drive/My Drive/models/cnn_revX.h5')

In [12]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(15, 15))

plt.subplot(2, 1, 1)
plt.title('Accuracy')
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.ylim([0,1.1])

plt.subplot(2, 1, 2)
plt.title('Loss')
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='center right')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy')
plt.ylim([0,3.5])

plt.show()


MIT License

Copyright (c) 2017 Fran√ßois Chollet

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.